In [14]:
import json
import pandas as pd

In [15]:
k = 3

data = pd.read_csv("generation\\database.csv")

numerical_qis = ['age']
categorical_qis = ['gender', 'city', 'education', 'profession']
all_qis = categorical_qis + numerical_qis
explicit_identifier = ['person_id', 'first_name', 'last_name']
sensitive_data = ['annual_income']
statistic = "range"

In [16]:
# Function to load JSON data from file
def load_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

In [17]:
# Function to drop specified columns from a DataFrame
def drop_EI(df, EI):
    return df.drop(columns=EI)

In [18]:
import json

def find_parents(node, target, path=None, key=None):
    """Trova tutti i genitori di un nodo target in un albero JSON"""
    if path is None:
        path = []

    current_name = node.get('name', None) if isinstance(node, dict) else None

    if isinstance(node, dict):
        # Caso in cui il nodo è un dizionario
        if key and node.get(key) == target:
            # Se è stato trovato il nodo target all'interno del dizionario
            return path + [current_name]

        for k, v in node.items():
            if isinstance(v, list) and target in v:
                # Se il nodo target è presente nella lista
                return path + [current_name]
            elif isinstance(v, (dict, list)):
                # Ricorsivamente cerca nei sotto-nodi
                result = find_parents(v, target, path + [current_name] if current_name else path, key)
                if result:
                    return result
    elif isinstance(node, list):
        # Caso in cui il nodo è una lista
        for item in node:
            if isinstance(item, (dict, list)):
                # Ricorsivamente cerca nei sotto-nodi
                result = find_parents(item, target, path, key)
                if result:
                    return result
            elif item == target:
                # Se il nodo target è stato trovato nella lista
                return path + [current_name]

    return None

def find_target_parents_in_json(file_path, target, key=None):
    """Trova tutti i genitori di un nodo target in un file JSON specificato. """
    with open(file_path, 'r') as file:
        data = json.load(file)

    return find_parents(data, target, key=key)


In [19]:
def find_lowest_common_ancestor(file_path, target1, target2, key=None):
    """Trova l'antenato comune più basso di due nodi target in un file JSON. """
    parents1 = find_target_parents_in_json(file_path, target1, key)
    parents2 = find_target_parents_in_json(file_path, target2, key)

    if not parents1 or not parents2:
        return None

    # Rimuove i valori None e deduplica le liste di genitori
    parents1 = [parent for parent in parents1 if parent]
    parents2 = [parent for parent in parents2 if parent]

    # Trova l'antenato comune più basso confrontando i percorsi
    lca = None
    for p1, p2 in zip(parents1, parents2):
        if p1 == p2:
            lca = p1
        else:
            break

    return lca

In [20]:
def splitter(dataframe, column, k):
    """Splits the dataframe along a certain column respecting k value"""
    if column in dataframe.select_dtypes(include='number').columns:
        median_value = dataframe[column].median()
        left_partition = dataframe[dataframe[column] <= median_value]
        right_partition = dataframe[dataframe[column] > median_value]
    elif column in dataframe.select_dtypes(include='object').columns:
        sorted_values = sorted(dataframe[column].unique())
        middle_index = len(sorted_values) // 2
        middle_value = sorted_values[middle_index]
        left_partition = dataframe[dataframe[column] <= middle_value]
        right_partition = dataframe[dataframe[column] > middle_value]
    else:
        # Skip columns that are neither numeric nor categorical
        return None, None
    
    if len(left_partition) >= k and len(right_partition) >= k:
        return left_partition, right_partition
    else:
        # Adjust partitions if lengths are less than k
        left_partition = dataframe.iloc[:k]
        right_partition = dataframe.iloc[k:]
        return left_partition, right_partition

In [21]:
# Esegui lo split per ogni colonna in all_qis
for column in all_qis:
    left, right = splitter(data, column, k)
    if left is not None and right is not None:
        print(f"Left partition shape for column '{column}': {left.shape}")
        print(f"Right partition shape for column '{column}': {right.shape}")

Left partition shape for column 'gender': (3, 9)
Right partition shape for column 'gender': (997, 9)
Left partition shape for column 'city': (544, 9)
Right partition shape for column 'city': (456, 9)
Left partition shape for column 'education': (737, 9)
Right partition shape for column 'education': (263, 9)
Left partition shape for column 'profession': (528, 9)
Right partition shape for column 'profession': (472, 9)
Left partition shape for column 'age': (510, 9)
Right partition shape for column 'age': (490, 9)


In [22]:
def recursive_partition(dataset, k, sensitive_data):
    """Splits the dataset in partitions recursively."""
    
    def axe_to_split(dataframe, sensitive_data):
        # Find column with highest cardinality (unique values) to split on
        columns_to_exclude = [col for col in dataframe.columns if col in sensitive_data]
        max_cardinality_column = dataframe.drop(columns_to_exclude, axis=1).nunique().idxmax()
        return max_cardinality_column
    
    # Base case: if dataset size is smaller than k*2, add it to partitions list
    if len(dataset) < k * 2:
        dataframe_partitions.append(dataset)
    else:
        # Split according to column with highest cardinality
        axe = axe_to_split(dataset, sensitive_data)
        left_partition, right_partition = splitter(dataset, axe, k)
        
        # Recursively partition left and right partitions
        recursive_partition(left_partition, k, sensitive_data)
        recursive_partition(right_partition, k, sensitive_data)

# Example usage:
dataframe_partitions = []

In [23]:
def generalize_partition(partition, qis, json_files, statistic):
    """Generalizza una partizione del dataset sostituendo i valori dei quasi-identificatori con la loro generalizzazione.
        Se il quasi-identificatore è numerico, viene generalizzato con il range di valori o la media.
        Se il quasi-identificatore è categorico, viene generalizzato con l'antenato comune più basso (LCA). """
    numerical_qis = [qi for qi in qis if partition[qi].dtype in ['int64', 'float64']]
    
    for qi in qis:
        # Ordina la partizione in base al quasi-identificatore corrente
        partition = partition.sort_values(by=qi)
        
        # Controlla se tutti i valori sono uguali per il quasi-identificatore corrente
        if partition[qi].iloc[0] != partition[qi].iloc[-1]:
            if qi in numerical_qis:
                if statistic == 'range':
                    # Se il quasi-identificatore è numerico, generalizza con il range di valori
                    min_val = partition[qi].iloc[0]
                    max_val = partition[qi].iloc[-1]
                    s = f"[{min_val}-{max_val}]"
                elif statistic == 'mean':
                    # Se il quasi-identificatore è numerico e la statistica è 'mean', generalizza con la media
                    mean_val = partition[qi].mean()
                    s = f"[{mean_val}]"
                else:
                    raise ValueError("Statistic must be 'range' or 'mean'")
            else:
                # Se il quasi-identificatore è categorico, cerca l'antenato comune più basso (LCA)
                unique_values = sorted(set(partition[qi]))
                if len(unique_values) == 1:
                    lca = unique_values[0]
                else:
                    lca = unique_values[0]
                    for value in unique_values[1:]:
                        lca_candidate = find_lowest_common_ancestor(json_files[qi], lca, value, key='name')
                        if lca_candidate:
                            lca = lca_candidate
                        else:
                            lca = 'ANY'
                            break
                        
                s = f"[{lca}]"
            
            # Sostituisce i valori del quasi-identificatore con la generalizzazione trovata
            partition[qi] = [s] * partition[qi].size
    
    return partition

In [24]:
json_files = {
    'city': 'generation/cities.json',
    'profession': 'generation/jobs.json',
    'education': 'generation/educations.json',
    'gender': 'generation/genders.json'
}

In [25]:
def mondrian(database, k, qis, sd, ei, json_files):
    global dataframe_partitions
    dataframe_partitions = []

    # Drop explicit identifiers from the database
    database = drop_EI(database, ei)
    
    # Perform recursive partitioning
    recursive_partition(database, k, sd)

    # Generalize partitions
    generalized_partitions = []
    for i, partition in enumerate(dataframe_partitions):
        generalized_partition = generalize_partition(partition, qis, json_files, statistic='range')
        generalized_partitions.append(generalized_partition)
    
    # Concatenate generalized partitions into anonymized_data
    anonymized_data = pd.concat(generalized_partitions, ignore_index=True)
    
    # Save anonymized data to CSV
    anonymized_data.to_csv('anonymized.csv', index=False)
    print("Dati anonimizzati salvati in anonymized.csv")
    
    # Print debugging information
    print(f"Numero di partizioni: {len(dataframe_partitions)}")
    total_rows = sum(len(partition) for partition in dataframe_partitions)
    print(f"Numero totale di righe nelle partizioni: {total_rows}")
    print(f"Numero originale di righe: {len(database)}")
    print(f"Numero totale di righe anonimizzate: {len(anonymized_data)}")


In [26]:
anonymized_data = mondrian(data, k, all_qis, sensitive_data, explicit_identifier, json_files)

Dati anonimizzati salvati in anonymized.csv
Numero di partizioni: 275
Numero totale di righe nelle partizioni: 1000
Numero originale di righe: 1000
Numero totale di righe anonimizzate: 1000
